
<div class="jumbotron">
  <h1><i class="fa fa-bar-chart" aria-hidden="true"></i> Integracción y limpieza</h1>
  <p></p>
</div>

In [615]:
from pandas import read_csv
from pandas import to_datetime
from pandas import DataFrame
from pandas import concat
from pickle import dump
import numpy as np

### Integración 

In [616]:
# Esta face se omite debido a que sólo es un archivos CSV
# Solo se toman el año 2021 por motivos de tiempo 

### Diccionario de datos


In [617]:
"""
Flujo Vehicular 2021

url: https://data.buenosaires.gob.ar/dataset/flujo-vehicular-por-radares-ausa/resource/2b37225d-ef92-4991-9045-f941f92a2f40

Flujo vehicular captado por radares desagregado por hora en 2021. 
Incluye fecha, hora, nombre de la autopista y ubicación del radar.

Campos del recurso

Nombre                     Tipo      Descripción
Mes, Día, Año de H_Fecha   string 	Mes, Día, Año de H_Fecha
H_Hora                    string 	H_Hora
Aut Nombre                string 	Aut Nombre
Disp Nombre               string 	Disp Nombre
Disp Ubicacion            string 	Disp Ubicacion
Seccion Sentido           string 	Seccion Sentido
Disp Lat                  string 	Disp Lat
Disp Lng                  string 	Disp Lng
H_Cant_Veh                string 	H_Cant_Veh  -> target

"""
h=0

### Cargar datos

In [618]:
df_2021 = read_csv('../Data/flujo-vehicular-por-radares-2021.csv', sep=';') 
df_2020 = read_csv('../Data/flujo-vehicular-por-radares-2020.csv', sep=',') 
df_2019 = read_csv('../Data/flujo-vehicular-por-radares-2019.csv', sep=',') 

/home/vagrant/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [619]:
df_2020.Fecha = to_datetime(df_2020.Fecha, format='%Y-%m-%d',errors ='coerce')
df_2019.Fecha = to_datetime(df_2019.Fecha, format='%Y-%m-%d',errors ='coerce')
df_2021.Fecha = to_datetime(df_2021.Fecha, format='%d/%m/%Y',errors ='raise')
#datos['Time'] = pd.to_datetime(datos['Time'], format='%Y-%m-%dT%H:%M:%SZ')

In [620]:


#df = concat([df_2019 ,df_2020,df_2021 ], axis=0)

In [621]:
df = df_2021

In [622]:
df.head(10)

,Fecha,Hora,Nombre_autopista,Nombre_dispositivo,Ubicación_dispositivo,Sentido,Latitud,Longitud,Num_vehiculos_hora
0,2021-01-01,0,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,943
1,2021-01-01,1,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,3946
2,2021-01-01,2,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,4662
3,2021-01-01,3,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,2486
4,2021-01-01,4,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,1268
5,2021-01-01,5,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,1195
6,2021-01-01,6,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,1092
7,2021-01-01,7,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,984
8,2021-01-01,8,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,719
9,2021-01-01,9,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,844


### Dimensiones del dataframe 

In [623]:
df.shape

(173348, 9)

### Features 

In [624]:
df.columns

Index(['Fecha', 'Hora', ' Nombre_autopista', 'Nombre_dispositivo',
       'Ubicación_dispositivo', 'Sentido', 'Latitud', 'Longitud',
       'Num_vehiculos_hora'],
      dtype='object')

In [625]:
#Cambiar el nombre de las variables

nombres = ['Fecha','Hora', 'Nombre_autopista','Nombre_dispositivo','Ubicación_dispositivo','Sentido','Latitud','Longitud', 'Num_vehiculos_hora']

df.columns = nombres

### Valores nulos o faltantes

In [626]:
df.isnull().sum()

Fecha                        0
Hora                         0
Nombre_autopista             0
Nombre_dispositivo           0
Ubicación_dispositivo        0
Sentido                    704
Latitud                  22663
Longitud                 22663
Num_vehiculos_hora           0
dtype: int64

### Valores nulos como porcentaje 

In [627]:
((df.isnull().sum() / len(df))*100).sort_values(ascending = False)

Latitud                  13.073701
Longitud                 13.073701
Sentido                   0.406119
Fecha                     0.000000
Hora                      0.000000
Nombre_autopista          0.000000
Nombre_dispositivo        0.000000
Ubicación_dispositivo     0.000000
Num_vehiculos_hora        0.000000
dtype: float64

<div class="alert alert-info" role="alert">
<ul>
<li> Existe un porcentaje alto de datos nulos dentro de las variables [Longitud,Latitud,Sentido]

<li> Debido a que son datos de sensores es común este comportamiento

<li> En esta etapa no es posible dedicir la forma de lidiar con los valores faltantes, por lo que se realizará un análisis exploratorio</ul>
</div>

### Analisis de valores faltantes

Tomaremos una sensor y una autopista y observaremos los datos

In [628]:
 df[(df.Nombre_dispositivo == "RD171 Esma") & (df.Nombre_autopista == "AU 4  Lugones")]

,Fecha,Hora,Nombre_autopista,Nombre_dispositivo,Ubicación_dispositivo,Sentido,Latitud,Longitud,Num_vehiculos_hora
0,2021-01-01,0,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,943
1,2021-01-01,1,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,3946
2,2021-01-01,2,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,4662
3,2021-01-01,3,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,2486
4,2021-01-01,4,AU 4 Lugones,RD171 Esma,9.9,A,NaN,-58.45967,1268
...,...,...,...,...,...,...,...,...,...
155967,2021-08-11,10,AU 4 Lugones,RD171 Esma,"9,9",A,-34.53767,-58.45967,8652
155968,2021-08-11,11,AU 4 Lugones,RD171 Esma,"9,9",A,-34.53767,-58.45967,7939
155969,2021-08-11,12,AU 4 Lugones,RD171 Esma,"9,9",A,-34.53767,-58.45967,7587
155970,2021-08-11,13,AU 4 Lugones,RD171 Esma,"9,9",A,-34.53767,-58.45967,7919



<div class="alert alert-info" role="alert">
<ul>
<li>Se pude observar que la latitud y longitud es la misma para todo el subconjunto de datos ya que se habla del mismo sensor, por lo cual es posible lidiar con los valores faltantes
</ul>
</div>

### Análisis de segmentos

 Primero obtendremos los valores unicos para cada variable de segmentación : autopista, sensor y sentido de circulación.

In [629]:
df.Sentido.unique()

array(['A', 'B', nan], dtype=object)

In [630]:
df.Nombre_autopista.unique()

array(['AU 4  Lugones', 'AU 5 Cantilo', 'AU 9 de Julio Sur',
       'AU Dellepiane', 'AU 7 Campora'], dtype=object)

In [631]:
df.Nombre_dispositivo.unique()

array(['RD171 Esma', 'RD170 Udaondo', 'RD169 Monroe', 'RD168 Juramento',
       'RD166 Ombues', 'RD165 Dorrego', 'RD183 Centro Naval',
       'RD182 Labruna', 'RD181 Ciudad Univer.', 'RD180 Encalada',
       'RD134 Ituzaingo', 'RD136 Yrigoyen', 'RD131 Brandsen B',
       'RD129 Olavarria', 'RD130 Suarez', 'RD128 Ben.Q.Martín B',
       'RD127 Ben.Q.Martín A', 'RD126 Osv.Cruz B', 'RD125 Osv.Cruz A',
       'RD100 Montiel', 'RD101 de la Torre', 'RD103 Miralla',
       'RD106 Zinny', 'RD110 Fernandez', 'RD108 Ramal a 27f.',
       'RD111 Ramal 9', 'RD190 Campora A', 'RD105 Homero'], dtype=object)

Agruparemos los datos por autopista y sensor

In [632]:
gb_df = df.groupby(['Nombre_autopista','Nombre_dispositivo'])

listar los valores de la segmentación

In [633]:
list((gb_df.groups.keys()))

[('AU 4  Lugones', 'RD165 Dorrego'),
 ('AU 4  Lugones', 'RD166 Ombues'),
 ('AU 4  Lugones', 'RD168 Juramento'),
 ('AU 4  Lugones', 'RD169 Monroe'),
 ('AU 4  Lugones', 'RD170 Udaondo'),
 ('AU 4  Lugones', 'RD171 Esma'),
 ('AU 5 Cantilo', 'RD180 Encalada'),
 ('AU 5 Cantilo', 'RD181 Ciudad Univer.'),
 ('AU 5 Cantilo', 'RD182 Labruna'),
 ('AU 5 Cantilo', 'RD183 Centro Naval'),
 ('AU 7 Campora', 'RD190 Campora A'),
 ('AU 9 de Julio Sur', 'RD125 Osv.Cruz A'),
 ('AU 9 de Julio Sur', 'RD126 Osv.Cruz B'),
 ('AU 9 de Julio Sur', 'RD127 Ben.Q.Martín A'),
 ('AU 9 de Julio Sur', 'RD128 Ben.Q.Martín B'),
 ('AU 9 de Julio Sur', 'RD129 Olavarria'),
 ('AU 9 de Julio Sur', 'RD130 Suarez'),
 ('AU 9 de Julio Sur', 'RD131 Brandsen B'),
 ('AU 9 de Julio Sur', 'RD134 Ituzaingo'),
 ('AU 9 de Julio Sur', 'RD136 Yrigoyen'),
 ('AU Dellepiane', 'RD100 Montiel'),
 ('AU Dellepiane', 'RD101 de la Torre'),
 ('AU Dellepiane', 'RD103 Miralla'),
 ('AU Dellepiane', 'RD105 Homero'),
 ('AU Dellepiane', 'RD106 Zinny'),
 ('A

número de segmentos:

In [634]:
len(list(gb_df.groups.keys()))

28

Ahora segmentaremos por autopista, sensor y sentido de circulación

In [635]:
gb_df_ = df.groupby(['Nombre_autopista','Nombre_dispositivo','Sentido'])
list((gb_df_.groups.keys()))

[('AU 4  Lugones', 'RD165 Dorrego', 'A'),
 ('AU 4  Lugones', 'RD166 Ombues', 'A'),
 ('AU 4  Lugones', 'RD168 Juramento', 'A'),
 ('AU 4  Lugones', 'RD169 Monroe', 'A'),
 ('AU 4  Lugones', 'RD170 Udaondo', 'A'),
 ('AU 4  Lugones', 'RD171 Esma', 'A'),
 ('AU 5 Cantilo', 'RD180 Encalada', 'B'),
 ('AU 5 Cantilo', 'RD181 Ciudad Univer.', 'B'),
 ('AU 5 Cantilo', 'RD182 Labruna', 'B'),
 ('AU 5 Cantilo', 'RD183 Centro Naval', 'B'),
 ('AU 7 Campora', 'RD190 Campora A', nan),
 ('AU 9 de Julio Sur', 'RD125 Osv.Cruz A', 'A'),
 ('AU 9 de Julio Sur', 'RD126 Osv.Cruz B', 'B'),
 ('AU 9 de Julio Sur', 'RD127 Ben.Q.Martín A', 'A'),
 ('AU 9 de Julio Sur', 'RD128 Ben.Q.Martín B', 'B'),
 ('AU 9 de Julio Sur', 'RD129 Olavarria', 'B'),
 ('AU 9 de Julio Sur', 'RD130 Suarez', 'A'),
 ('AU 9 de Julio Sur', 'RD131 Brandsen B', 'B'),
 ('AU 9 de Julio Sur', 'RD134 Ituzaingo', 'A'),
 ('AU 9 de Julio Sur', 'RD134 Ituzaingo', 'B'),
 ('AU 9 de Julio Sur', 'RD136 Yrigoyen', 'A'),
 ('AU 9 de Julio Sur', 'RD136 Yrigoyen', '

In [636]:
#unicos por grupo de lat,long
#for group_name, df_group in gb_df:
#    print(group_name)
#    print(df_group.head(20))

In [637]:
#gb_df.get_group(('AU 7 Campora', 'RD190 Campora A', None))

In [638]:
#unicos por grupo de lat,long
#for group_name, df_group in gb_df:
#    print(group_name, ':::','Size: ',df_group.size )
#    print(group_name, ':::','Size: ',df_group.isnull().sum() )

In [639]:
len(list(gb_df.groups.keys()))

28

<div class="alert alert-info" role="alert">
<ul>
<li> Se obtiene el mismo número de segmentos, lo cual indica que los sensores sólo atienden un sentido de circulación, y es posible lidiar con los valores daltantes de la variable 'sentido'</li>
    </ul>
</div>

Se creara un diccionario para la latitud y longitud de cada segmento, con lo cual se podra lidiar con los datos faltantes

In [640]:
my_dict = {}

In [641]:
#unicos por grupo de lat,long
for group_name, df_group in gb_df:
    #print(group_name)
    lat = df_group.Latitud.unique()
    long_ = df_group.Longitud.unique()
    print(lat)
    print(long_)
    lat = [x for x in lat if str(x) != 'nan']
    long_ = [x for x in long_ if str(x) != 'nan']
    print(lat)
    print(long_)
    lat = [sub.replace(',', '.') if isinstance(sub, str) else sub for sub in lat ]
    long_ = [sub.replace(',', '.')if isinstance(sub, str) else sub for sub in long_ ]
    
    lat = list(set(map(float, lat)))
    long_ = list(set(map(float, long_)))
    print(lat)
    print(long_)
    my_dict[group_name]={'lat':lat[0],'long_':long_[0]}
    #print(lat)
    #print(long_)

[nan -34.56336 '-34.56336' '-34,56336']
[-58.41529 nan '-58.41529' '-58,41529']
[-34.56336, '-34.56336', '-34,56336']
[-58.41529, '-58.41529', '-58,41529']
[-34.56336]
[-58.41529]
[nan -34.56044 '-34.56044' '-34,56044']
[-58.41908000000001 nan '-58.41908000000001' '-58,41908' '-58.41908']
[-34.56044, '-34.56044', '-34,56044']
[-58.41908000000001, '-58.41908000000001', '-58,41908', '-58.41908']
[-34.56044]
[-58.41908, -58.41908000000001]
[nan -34.54977 '-34.54977' '-34,54977']
[-58.43601999999999 nan '-58.43601999999999' '-58,43602' '-58.43602']
[-34.54977, '-34.54977', '-34,54977']
[-58.43601999999999, '-58.43601999999999', '-58,43602', '-58.43602']
[-34.54977]
[-58.43602, -58.43601999999999]
[nan -34.54629 '-34.54629' '-34,54629']
[-58.44224000000001 nan '-58.44224000000001' '-58,44224' '-58.44224']
[-34.54629, '-34.54629', '-34,54629']
[-58.44224000000001, '-58.44224000000001', '-58,44224', '-58.44224']
[-34.54629]
[-58.44224, -58.44224000000001]
[nan -34.54293 '-34.54293' '-34,54293

Ejemplo de acceso a la latitud del segmento ('AU 4  Lugones', 'RD165 Dorrego')

In [642]:
my_dict[('AU 4  Lugones', 'RD165 Dorrego')]['lat']

-34.56336

remplazo de valores NaN de cada segmento

In [643]:
new_df =  DataFrame() 

In [644]:
# iterate over each group
for group_name, df_group in gb_df:
    print(group_name)
    df_group.Latitud.replace(np.nan,my_dict[group_name]['lat'],inplace=True )
    df_group.Longitud.replace(np.nan,my_dict[group_name]['long_'],inplace=True )
    
    #remplazo de los valores faltantes de la variable sentido
    if group_name == ('AU 7 Campora', 'RD190 Campora A'):
        df_group.Sentido.replace(np.nan,'A',inplace=True )
    new_df= concat([df_group,new_df])
    #print(df_group.size)

('AU 4  Lugones', 'RD165 Dorrego')
('AU 4  Lugones', 'RD166 Ombues')
('AU 4  Lugones', 'RD168 Juramento')
('AU 4  Lugones', 'RD169 Monroe')
('AU 4  Lugones', 'RD170 Udaondo')
('AU 4  Lugones', 'RD171 Esma')
('AU 5 Cantilo', 'RD180 Encalada')
('AU 5 Cantilo', 'RD181 Ciudad Univer.')
('AU 5 Cantilo', 'RD182 Labruna')
('AU 5 Cantilo', 'RD183 Centro Naval')
('AU 7 Campora', 'RD190 Campora A')
('AU 9 de Julio Sur', 'RD125 Osv.Cruz A')
('AU 9 de Julio Sur', 'RD126 Osv.Cruz B')
('AU 9 de Julio Sur', 'RD127 Ben.Q.Martín A')
('AU 9 de Julio Sur', 'RD128 Ben.Q.Martín B')
('AU 9 de Julio Sur', 'RD129 Olavarria')
('AU 9 de Julio Sur', 'RD130 Suarez')
('AU 9 de Julio Sur', 'RD131 Brandsen B')
('AU 9 de Julio Sur', 'RD134 Ituzaingo')
('AU 9 de Julio Sur', 'RD136 Yrigoyen')
('AU Dellepiane', 'RD100 Montiel')
('AU Dellepiane', 'RD101 de la Torre')
('AU Dellepiane', 'RD103 Miralla')
('AU Dellepiane', 'RD105 Homero')
('AU Dellepiane', 'RD106 Zinny')
('AU Dellepiane', 'RD108 Ramal a 27f.')
('AU Dellepian

In [645]:
new_df

,Fecha,Hora,Nombre_autopista,Nombre_dispositivo,Ubicación_dispositivo,Sentido,Latitud,Longitud,Num_vehiculos_hora
36410,2021-01-01,0,AU Dellepiane,RD111 Ramal 9,0.18,B,-34.65061,-58.46677,60
36411,2021-01-01,1,AU Dellepiane,RD111 Ramal 9,0.18,B,-34.65061,-58.46677,142
36412,2021-01-01,2,AU Dellepiane,RD111 Ramal 9,0.18,B,-34.65061,-58.46677,228
36413,2021-01-01,3,AU Dellepiane,RD111 Ramal 9,0.18,B,-34.65061,-58.46677,130
36414,2021-01-01,4,AU Dellepiane,RD111 Ramal 9,0.18,B,-34.65061,-58.46677,76
...,...,...,...,...,...,...,...,...,...
153277,2021-08-31,19,AU 4 Lugones,RD165 Dorrego,"4,74",A,-34.56336,-58.41529,4348
153278,2021-08-31,20,AU 4 Lugones,RD165 Dorrego,"4,74",A,-34.56336,-58.41529,3628
153279,2021-08-31,21,AU 4 Lugones,RD165 Dorrego,"4,74",A,-34.56336,-58.41529,2322
153280,2021-08-31,22,AU 4 Lugones,RD165 Dorrego,"4,74",A,-34.56336,-58.41529,1624


### Valores duplicados

In [646]:
new_df.sort_values(['Fecha','Hora'])

,Fecha,Hora,Nombre_autopista,Nombre_dispositivo,Ubicación_dispositivo,Sentido,Latitud,Longitud,Num_vehiculos_hora
36410,2021-01-01,0,AU Dellepiane,RD111 Ramal 9,0.18,B,-34.65061,-58.46677,60
44582,2021-01-01,0,AU Dellepiane,RD111 Ramal 9,0.18,B,-34.65061,-58.46677,60
34922,2021-01-01,0,AU Dellepiane,RD110 Fernandez,4.3,B,-34.65288,-58.46818,8
43838,2021-01-01,0,AU Dellepiane,RD110 Fernandez,4.3,B,-34.65288,-58.46818,8
35666,2021-01-01,0,AU Dellepiane,RD108 Ramal a 27f.,4.3,A,-34.65369,-58.46812,121
...,...,...,...,...,...,...,...,...,...
156713,2021-08-31,23,AU 5 Cantilo,RD180 Encalada,"7,4",B,-34.54778,-58.43848,634
155716,2021-08-31,23,AU 4 Lugones,RD170 Udaondo,"8,68",A,-34.54293,-58.44901,1719
154767,2021-08-31,23,AU 4 Lugones,RD168 Juramento,"7,2",A,-34.54977,-58.43602,1617
154025,2021-08-31,23,AU 4 Lugones,RD166 Ombues,"5,5",A,-34.56044,-58.41908,1656


In [647]:
new_df.drop_duplicates( keep='first',inplace=True)

In [648]:
new_df.sort_values(['Fecha','Hora'])

,Fecha,Hora,Nombre_autopista,Nombre_dispositivo,Ubicación_dispositivo,Sentido,Latitud,Longitud,Num_vehiculos_hora
36410,2021-01-01,0,AU Dellepiane,RD111 Ramal 9,0.18,B,-34.65061,-58.46677,60
34922,2021-01-01,0,AU Dellepiane,RD110 Fernandez,4.3,B,-34.65288,-58.46818,8
35666,2021-01-01,0,AU Dellepiane,RD108 Ramal a 27f.,4.3,A,-34.65369,-58.46812,121
43094,2021-01-01,0,AU Dellepiane,RD108 Ramal a 27f.,4.3,A,-34.65369,-58.46812,121
33434,2021-01-01,0,AU Dellepiane,RD106 Zinny,3.88,A,-34.65465,-58.46901,537
...,...,...,...,...,...,...,...,...,...
156713,2021-08-31,23,AU 5 Cantilo,RD180 Encalada,"7,4",B,-34.54778,-58.43848,634
155716,2021-08-31,23,AU 4 Lugones,RD170 Udaondo,"8,68",A,-34.54293,-58.44901,1719
154767,2021-08-31,23,AU 4 Lugones,RD168 Juramento,"7,2",A,-34.54977,-58.43602,1617
154025,2021-08-31,23,AU 4 Lugones,RD166 Ombues,"5,5",A,-34.56044,-58.41908,1656


### Eliminar datos nulos

In [649]:
# No aplica

### Verificar datos 

In [650]:
new_df.describe()

,Hora,Num_vehiculos_hora
count,156511.000000,156511.000000
mean,11.502891,2226.582426
std,6.921794,1985.933472
min,0.000000,0.000000
25%,6.000000,564.000000
50%,12.000000,1667.000000
75%,18.000000,3377.000000
max,23.000000,9696.000000


In [651]:
#Datos nulos
((new_df.isnull().sum() / len(new_df))*100).sort_values(ascending = False)

Fecha                    0.0
Hora                     0.0
Nombre_autopista         0.0
Nombre_dispositivo       0.0
Ubicación_dispositivo    0.0
Sentido                  0.0
Latitud                  0.0
Longitud                 0.0
Num_vehiculos_hora       0.0
dtype: float64

### Transformar columna `Data` a formato datetime

In [652]:
new_df.Fecha = to_datetime(new_df.Fecha, format='%d/%m/%Y')
#datos['Time'] = pd.to_datetime(datos['Time'], format='%Y-%m-%dT%H:%M:%SZ')

### Obtener el nombre de las variables categorical

In [653]:
### No aplica

### Convertir las variables a tipo `category`

In [654]:
# NO aplica

### Almacenar el `dataframe` en formato pickle 

In [655]:
new_df

,Fecha,Hora,Nombre_autopista,Nombre_dispositivo,Ubicación_dispositivo,Sentido,Latitud,Longitud,Num_vehiculos_hora
36410,2021-01-01,0,AU Dellepiane,RD111 Ramal 9,0.18,B,-34.65061,-58.46677,60
36411,2021-01-01,1,AU Dellepiane,RD111 Ramal 9,0.18,B,-34.65061,-58.46677,142
36412,2021-01-01,2,AU Dellepiane,RD111 Ramal 9,0.18,B,-34.65061,-58.46677,228
36413,2021-01-01,3,AU Dellepiane,RD111 Ramal 9,0.18,B,-34.65061,-58.46677,130
36414,2021-01-01,4,AU Dellepiane,RD111 Ramal 9,0.18,B,-34.65061,-58.46677,76
...,...,...,...,...,...,...,...,...,...
153277,2021-08-31,19,AU 4 Lugones,RD165 Dorrego,"4,74",A,-34.56336,-58.41529,4348
153278,2021-08-31,20,AU 4 Lugones,RD165 Dorrego,"4,74",A,-34.56336,-58.41529,3628
153279,2021-08-31,21,AU 4 Lugones,RD165 Dorrego,"4,74",A,-34.56336,-58.41529,2322
153280,2021-08-31,22,AU 4 Lugones,RD165 Dorrego,"4,74",A,-34.56336,-58.41529,1624


In [656]:
# Store data 
with open('../Data/df.pickle', 'wb') as handle:
    dump(new_df, handle)